In [ ]:
import pandas as pd
import numpy as np
from numpy import median, average
import seaborn as sns
import matplotlib.pyplot as plt

import sqlite3  
from sqlalchemy import create_engine, select, MetaData, Table, Integer, String, inspect, Column, ForeignKey
import os

pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 300)
pd.set_option('display.width', 1000)

In [ ]:
import altair as alt

In [ ]:
cali = pd.read_csv('statewide_cases.csv')  
pop = pd.read_csv('county_pop.csv')

In [ ]:
pop = pop.rename(columns={'County': 'county', "Population": "population"})
pop

In [ ]:
cali

In [ ]:
cali.info()

In [ ]:
cali.describe(include=np.object)

In [ ]:
cali.describe()

# What we do know?

## - there are 60 Unique Counties in Cali 

## - there are 137 unique days (4.5 months)

In [ ]:
cali_numbers = cali[['date','county','newcountconfirmed', 'newcountdeaths']]

In [ ]:
top_10 = cali_numbers.groupby(['county'], as_index=False).sum().sort_values(['newcountconfirmed', 'newcountdeaths'], ascending=False)
top = top_10.nlargest(10, "newcountconfirmed")
top = top[['county']]
top

In [ ]:
cali_numbers.groupby(['county','date'], as_index=False).sum().sort_values(['newcountconfirmed', 'newcountdeaths'], ascending=False)
cali_total = cali_numbers.melt(id_vars=["county","date"], var_name="Type", value_name="Value")
cali_total['date'] = pd.to_datetime(cali_total['date'])
cali_total['Value'] = cali_total['Value'].astype(float)
cali_total

In [ ]:
cali_top_10 = cali_total[cali_total['county'].isin(top['county'])]
cali_top_10

In [53]:
la_ala = cali_top_10[cali_top_10['county'].isin(['Alameda', 'Los Angeles'])]
la_ala['county'] =  la_ala['county'] + ' County'
la_ala = la_ala.merge(pop, how='left', on=['county'])
la_ala['per_capita'] = la_ala['Value'] / la_ala['population'] * 100000
la_ala['SMA_7'] = 0
alameda_case = la_ala[(la_ala['Type'] == 'newcountconfirmed') & (la_ala['county'] == 'Alameda County')]
alameda_death = la_ala[(la_ala['Type'] == 'newcountdeaths') & (la_ala['county'] == 'Alameda County')]
la_case = la_ala[(la_ala['Type'] == 'newcountconfirmed') & (la_ala['county'] == 'Los Angeles County')]
la_death = la_ala[(la_ala['Type'] == 'newcountdeaths') & (la_ala['county'] == 'Los Angeles County')]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [55]:
def simple_moving_ave(df):
    for i in range (0, df.shape[0]-6):
        df.loc[df.index[i+6],'SMA_7'] = np.round(((df.iloc[i,5] + df.iloc[i+1,5] + df.iloc[i+2,5] + df.iloc[i+3,5] + df.iloc[i+4,5] + df.iloc[i+5,5] + df.iloc[i+6,5])/7),5)
    return df
    
sma = [simple_moving_ave(alameda_case), simple_moving_ave(alameda_death), simple_moving_ave(la_case), simple_moving_ave(la_death)]
       
result = pd.concat(sma)
result


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,county,date,Type,Value,population,per_capita,SMA_7
0,Alameda County,2020-03-18,newcountconfirmed,29.0,1671329,1.735146,0.00000
1,Alameda County,2020-03-19,newcountconfirmed,7.0,1671329,0.418828,0.00000
2,Alameda County,2020-03-20,newcountconfirmed,6.0,1671329,0.358996,0.00000
3,Alameda County,2020-03-21,newcountconfirmed,20.0,1671329,1.196652,0.00000
4,Alameda County,2020-03-22,newcountconfirmed,10.0,1671329,0.598326,0.00000
5,Alameda County,2020-03-23,newcountconfirmed,16.0,1671329,0.957322,0.00000
6,Alameda County,2020-03-24,newcountconfirmed,8.0,1671329,0.478661,0.82056
7,Alameda County,2020-03-25,newcountconfirmed,38.0,1671329,2.273640,0.89749
8,Alameda County,2020-03-26,newcountconfirmed,38.0,1671329,2.273640,1.16246
9,Alameda County,2020-03-27,newcountconfirmed,13.0,1671329,0.777824,1.22230


In [ ]:
all_cali_date = cali_total.groupby(['date'], as_index=False).sum().sort_values(['newcountconfirmed', 'newcountdeaths'], ascending=False)
all_cali_date = all_cali_date.melt(id_vars=["date"], var_name="Type", value_name="Value")
all_cali_date['date'] = pd.to_datetime(all_cali_date['date'])
all_cali_date['Value'] = all_cali_date['Value'].astype(float)
all_cali_date

In [ ]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')

line = alt.Chart(all_cali_date).mark_line(point=True).encode(
    x = alt.X('date', axis = alt.Axis(title = 'date'.upper(), format = ("%b %Y"), tickMinStep = 2, labelAngle=0)),
    y=alt.Y('Value', axis = alt.Axis(title='Value')),
    color='Type',
)# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(all_cali_date).mark_point().encode(
    x='date',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Value', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(all_cali_date).mark_rule(color='gray').encode(
    x='date'
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=1200,
    height=600
).configure_axis(
    labelFontSize=20,
    titleFontSize=20,
).configure_legend(
    titleFontSize=10,
    labelFontSize=15,
).interactive()

In [68]:
alt.data_transformers.disable_max_rows()
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')

line = alt.Chart(result, title="7-Day Moving Average LA vs Alameda COVID-19").mark_line(point=True).encode(
    x = alt.X('date:T', axis = alt.Axis(title = 'date'.upper(), format = ("%b %Y"), tickMinStep = 2, labelAngle=0)),
    y=alt.Y('SMA_7', axis = alt.Axis(title='Per 100,000 Population')),
    color='county',
    strokeDash='Type',
)# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(result).mark_point().encode(
    x='date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'SMA_7', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(result).mark_rule(color='gray').encode(
    x='date'
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=900,
    height=600
).configure_axis(
    labelFontSize=20,
    titleFontSize=20,
).configure_legend(
    titleFontSize=10,
    labelFontSize=15,
).configure_title(fontSize=24).interactive()

alt.LayerChart(...)

## Additional Demographics

In [ ]:
age = pd.read_csv('case_demographics_age.csv')  
race = pd.read_csv('case_demographics_ethnicity.csv')
sex = pd.read_csv('case_demographics_sex.csv')
homeless = pd.read_csv('homeless_impact.csv')

In [ ]:
age

In [ ]:
sex

In [ ]:
race.sort_values(by=["deaths"], ascending=False)

In [ ]:
homeless.sort_values(by=['county', 'date'])